In [4]:
import tarfile
import sqlite3
import csv
import pandas as pd


In [36]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName('Large tar.gz Processing').getOrCreate()

# Read data directly using Spark's capabilities
df = spark.read.csv('20181113_training_set.tar.gz')
df.show(10)

# Print the list of columns
print(df.columns)

# Stop the Spark session
spark.stop()

+--------------------+----------------+--------------+--------------------+------+------+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+-------+---------------+--------------------+--------------------+
|                 _c0|             _c1|           _c2|                 _c3|   _c4|   _c5|   _c6|        _c7|           _c8|                 _c9|                _c10|                _c11|                _c12|                _c13|                _c14|       _c15|      _c16|   _c17|           _c18|                _c19|                _c20|
+--------------------+----------------+--------------+--------------------+------+------+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+-------+---------------+--------------------+-----

In [29]:
!ls extracted/training_set/


log_0_20180815_000000000000.csv log_4_20180816_000000000000.csv
log_2_20180908_000000000000.csv log_5_20180809_000000000000.csv
log_3_20180829_000000000000.csv log_9_20180820_000000000000.csv
log_4_20180719_000000000000.csv


In [48]:
df = pd.read_csv(filepath_or_buffer="training_set/log_0_20180716_000000000000.csv")
df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,1,20,t_91eb6576-0f1f-4394-92d8-560d7b8f632b,True,True,True,False,0,0,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,fwdbtn
1,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,2,20,t_338cb0f3-0035-418f-ab2d-3f9342815aaa,True,True,True,False,0,1,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,fwdbtn
2,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,3,20,t_98e73634-8f33-4275-b544-3451553f431e,False,True,True,False,0,1,...,0,0,0,True,5,2018-07-16,True,editorial_playlist,fwdbtn,endplay
3,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,4,20,t_2fd6279a-06cd-4d5d-9e54-1912f51bb3c3,False,False,False,True,0,1,...,0,0,0,False,6,2018-07-16,True,editorial_playlist,clickrow,trackdone
4,1_00001238-3f7a-4fd2-b52e-af4237a41ed9,5,20,t_de860a61-bb57-4386-9d5a-6ba111c4c266,False,False,False,True,0,1,...,0,0,0,False,6,2018-07-16,True,editorial_playlist,trackdone,trackdone


In [49]:
len(df)

3392315

In [50]:
df["session_id"].nunique()

203225

In [39]:
import tarfile
import os

def process_and_count_rows(tar_path, extract_temp_path):
    # Dictionary to hold file names and their row counts
    file_row_counts = {}

    with tarfile.open(tar_path, 'r:gz') as tar:
        # Iterate over each member in the tar file
        for member in tar.getmembers():
            if member.name.endswith('.csv'):
                # Extract the file
                tar.extract(member, path=extract_temp_path)
                file_path = os.path.join(extract_temp_path, member.name)

                # Load the CSV file into a DataFrame
                df = spark.read.csv(file_path, header=True, inferSchema=True)

                # Count the number of rows in the DataFrame
                row_count = df.count()

                # Store the count in the dictionary
                file_row_counts[member.name] = row_count

                # Optionally, delete the file after processing if disk space is a concern
                os.remove(file_path)

    return file_row_counts

# Specify paths
tar_path = '20181113_training_set.tar.gz'
extract_temp_path = 'extracted_folder'

# Process the tar.gz file and get row counts for each CSV file
row_counts = process_and_count_rows(tar_path, extract_temp_path)
print("Row counts per file:", row_counts)

# Stop the Spark session
spark.stop()


Py4JJavaError: An error occurred while calling o127.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConne